# Estimate diffusivity parameters from 1D b-space optimization
## Ryan Holmes

In [ ]:
# Load modules
import numpy as np
import h5py
import matplotlib
import shutil
import os
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
matplotlib.rcParams["figure.facecolor"] = "white"
matplotlib.rcParams["axes.facecolor"] = "white"
matplotlib.rcParams["savefig.facecolor"] = "white"
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import time
from IPython import display
import obsfit1d
from scipy.interpolate import interp1d

In [ ]:
# 2D run data:
basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns24-5-19/';
run = [basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_400/'];
params = np.load(run[0] + 'runparams.npz');locals().update(params)
b = np.linspace(0,N2*np.cos(theta)*Lz + N2*np.sin(theta)*Ly,64);db = b[1]-b[0]
ba = (b[:-1]+b[1:])/2

with h5py.File(run[0] + 'ifields.h5', mode='r') as file:
    B = file['tasks']['B'][0,:,:]
    B = (B[1:,:] + B[:-1,:])/2
    B = (B[:,1:] + B[:,:-1])/2
    y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
dy = y[1:]-y[:-1];dz = z[1:]-z[:-1];
dA = np.tile(dy,[len(z)-1,1]).T*np.tile(dz,[len(y)-1,1]);

with h5py.File(run[0] + 'snapshots.h5', mode='r') as file:

    t = file['scales/sim_time'][:]/lday;
    trT = file['tasks']['trT'][:,0,0]
    trF = np.zeros((len(b)-1,len(t)))
    for i in range(0,len(t)):#len(t),pfr):
        tr = file['tasks']['tr'][i,:,:]
        tr = (tr[1:,:] + tr[:-1,:])/2
        tr = (tr[:,1:] + tr[:,:-1])/2
        tr = tr*dA
        trF[:,i] = [np.sum(tr[np.logical_and(B>=bi,B<(bi+db))])/db for bi in b[:-1]]

zF = (b[1:]+b[:-1])/2/N2;
dzF = zF[1]-zF[0]
tF = t;

imax = np.argmax(trF[:,0])
zF = zF - zF[imax]
trF = trF/np.sum(trF[imax,0]*dzF)

In [ ]:
import imp
imp.reload(obsfit1d)

In [ ]:
res = obsfit1d.fit3par(zF,trF[:,-1],int(tF[-1]*1e5),sz)

# Plotting for check:
Lh = 4000. # Max size of h-grid
n = 200 # Number of h-points
dt = int(1e5) # Time-step
h = np.linspace(-Lh/2.,Lh/2.,n)
dh = h[1]-h[0]

trI = np.exp(-h**2/2/sz**2)
trI = trI / np.sum(trI*dh)

# Observed tracer distribution on grid:
f = interp1d(zF,trF[:,-1],fill_value=0.)
trO = np.zeros_like(h)
in_range = np.logical_and(h>=np.min(zF),h <= np.max(zF))
trO[in_range] = f(h[in_range])
trO = trO / np.sum(trO*dh)

A = obsfit1d.operator(res.x[0],res.x[1],res.x[2])
trM = obsfit1d.solve(A,int(tF[-1]*1e5) // dt, trI)

f = plt.figure(figsize=(7.5,5),facecolor='white')
#plt.plot(trI,h,'-r')
plt.plot(trO,h,'-k')
plt.plot(trM,h,'-b')
print(res.x)


In [ ]:
# Input h Grid
Lh = 2000.;n = 200
h = np.linspace(-Lh/2.,Lh/2.,n)
dh = h[1]-h[2]

# Initial Condition:
#sh = sz; # from 2D problem Initial condition.
sh = Lh/20.;
ch = 0.;
trI = np.exp(-(h-ch)**2/2/sh**2)/np.sqrt(2*np.pi*sh**2)

# Time information:
dt = 1.e5 # seconds
nt = 2000 # Number of iterations

# Input parameters
w = 0.0#1.0e-6
K0 = 2e-4
Kh = 1.e-7

# Solve:
tr = Solve1D.solve(Lh,n,dt,nt,K0,Kh,w,trI)
print(Solve1D.chisq(tr,trI))

plt.plot(trI,h)
plt.plot(tr,h,'-r')